<a href="https://colab.research.google.com/github/aisyahdnn/Skripsi/blob/master/EDataset1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from skimage import io
from skimage import feature

In [3]:
Citra = os.listdir('/content/drive/My Drive/Dataset/DataTest1')
print(Citra)
print(len(Citra))

['C37BP2_thinF_IMG_20150620_132440a_cell_108.png', 'C37BP2_thinF_IMG_20150620_132847a_cell_76.png']
2


In [0]:
#SEGMENTASI CANNY DAN EKSTRAKSI HUMOMENT

x = np.array([['H1','H2','H3','H4','H5','H6','H7','Actual']])

#Proses Citra Malaria Terinfeksi
for i in range(len(Citra)):
    img = cv2.imread('/content/drive/My Drive/Dataset/DataTest1'+ "/" + Citra[i])
    edges = cv2.Canny(img,80,30)
    a = cv2.HuMoments(cv2.moments(edges)).flatten()
    a = np.append(a,1)
    x = np.vstack((x,a))

In [0]:
#EXPORT to CSV
np.savetxt("/content/drive/My Drive/Dataset/EDatasetTest1.csv", x, fmt='%s',delimiter=',' )

In [18]:
#LOAD DATASET

datasettrain = pd.read_csv('/content/drive/My Drive/Dataset/EDatasetTrain.csv')
datasettest = pd.read_csv('/content/drive/My Drive/Dataset/EDatasetTest1.csv')

print(datasettrain)
print(datasettest)

           H1        H2            H3  ...            H6            H7  Actual
0    0.014561  0.000029  3.632756e-08  ...  6.353503e-12  4.330040e-16     1.0
1    0.026547  0.000170  1.138837e-06  ...  3.063264e-08 -3.005370e-12     1.0
2    0.028276  0.000027  3.460565e-07  ...  5.265115e-09 -6.515198e-13     1.0
3    0.024422  0.000022  1.157081e-07  ...  3.187912e-10 -6.131459e-14     1.0
4    0.019319  0.000002  1.072033e-07  ...  7.293107e-10  9.598053e-14     1.0
..        ...       ...           ...  ...           ...           ...     ...
995  0.012752  0.000043  2.179522e-07  ...  1.653583e-11 -1.029641e-16     1.0
996  0.018757  0.000024  4.702513e-09  ...  6.308761e-09  1.009837e-13     1.0
997  0.018091  0.000002  7.336674e-08  ...  3.725437e-10  9.225196e-14     1.0
998  0.019440  0.000045  1.845630e-07  ...  2.614035e-09 -1.054526e-13     1.0
999  0.016831  0.000017  1.238836e-07  ... -2.620264e-12 -3.714716e-17     1.0

[1000 rows x 8 columns]
         H1            H2  

In [0]:
#split dataset

x_train = datasettrain.iloc[:, 0:7] 
y_train = datasettrain.iloc[:, 7]
 
x_test = datasettest.iloc[:, 0:7] 
y_test = datasettest.iloc[:, 7]

In [20]:
#scaling data (menyamakan skala nilainya)
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)

x_test = sc_x.transform(x_test)
x_train

array([[-1.03210576e+00,  5.39451025e-02, -5.43397287e-01, ...,
         5.02095747e-02, -2.94088350e-01,  5.38971312e-04],
       [ 1.30856135e-01,  3.54579075e+00,  1.20175903e+00, ...,
         2.90920972e+00,  6.78844865e+00, -5.15607935e+00],
       [ 2.98553462e-01,  1.32841329e-02, -5.31289080e-02, ...,
         2.67654431e-02,  9.22036037e-01, -1.11792161e+00],
       ...,
       [-6.89622593e-01, -6.08502851e-01, -4.84768160e-01, ...,
         1.20976730e-01, -2.09404378e-01,  1.58059380e-01],
       [-5.58685644e-01,  4.71373119e-01, -3.08756258e-01, ...,
        -1.88137262e-02,  3.08955739e-01, -1.81113531e-01],
       [-8.11885477e-01, -2.23388664e-01, -4.04805268e-01, ...,
         5.02282559e-02, -2.96163594e-01, -2.67598526e-04]])

In [21]:
akurasi = 0
for x in range(2,10): 
    #choose method and fitting
    classifier = KNeighborsClassifier(n_neighbors=x,p=2,metric='euclidean')
    classifier.fit(x_train, y_train)

    #testing data
    y_pred = classifier.predict(x_test)

    #print result
    cm = confusion_matrix(y_test, y_pred)
    tertinggi = accuracy_score(y_test, y_pred)
    if tertinggi >= akurasi:
        akurasi = tertinggi
        cmx = cm
        k = x
        a = accuracy_score(y_test, y_pred)
        p = precision_score(y_test, y_pred)
        r = recall_score(y_test, y_pred)
        f = f1_score(y_test, y_pred)
print("K :" , k)
print(cmx)
print("Akurasi :" , a)
print("presisi :" , p)
print("recall :" , r)
print("F-Score :" , f)
print("--------------------------------------------")

K : 9
[[2]]
Akurasi : 1.0
presisi : 1.0
recall : 1.0
F-Score : 1.0
--------------------------------------------
